In [13]:
import sys
# !{sys.executable} -m pip install torch torchvision
# !{sys.executable} -m pip install pandas
!{sys.executable} -m pip install -U scikit-learn

  Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Using cached scipy-1.15.2-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl (11.1 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.15.2-cp312-cp312-win_amd64.whl (40.9 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)



[notice] A new release of pip is available: 24.2 -> 25.1
[notice] To update, run: C:\Users\rosha\Documents\Jupyter\NIDS\gan_venv\gan-venv\Scripts\python.exe -m pip install --upgrade pip


In [4]:
import sys
print(sys.executable)

C:\Users\rosha\Documents\Jupyter\NIDS\gan_venv\gan-venv\Scripts\python.exe


In [3]:
!pip list

Package                   Version
------------------------- --------------
absl-py                   2.1.0
annotated-types           0.7.0
anyio                     4.6.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.4.1
astunparse                1.6.3
async-lru                 2.0.4
attrs                     24.2.0
babel                     2.16.0
beautifulsoup4            4.12.3
bleach                    6.1.0
certifi                   2024.8.30
cffi                      1.17.1
charset-normalizer        3.3.2
colorama                  0.4.6
comm                      0.2.2
contourpy                 1.3.0
cycler                    0.12.1
debugpy                   1.8.5
decorator                 5.1.1
defusedxml                0.7.1
executing                 2.1.0
fastjsonschema            2.20.0
filelock                  3.18.0
flatbuffers               24.3.25
fonttools                 4.54.1
fqdn         

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler


In [1]:
data_path = 'F:/Documents/CRCE/Project/NIDS/dataset/Edge-IIoT/Edge-IIoTset dataset/Selected dataset for ML and DL/DNN-EdgeIIoT-dataset.csv'  # Replace with your actual path

In [6]:
# Generator Model
class Generator(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim),
        )

    def forward(self, z):
        return self.model(z)

# Discriminator Model
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)


In [5]:
def preprocess_edge_iiot(filepath):
    df = pd.read_csv(filepath, low_memory=False)

    # Drop non-useful columns
    cols_to_drop = ['Attack_type', 'frame.time']  # if present
    for col in cols_to_drop:
        if col in df.columns:
            df = df.drop(columns=col)

    # Encode 'Attack_label' separately
    if 'Attack_label' in df.columns:
        y = df['Attack_label'].values
        df = df.drop(columns=['Attack_label'])
    else:
        y = None

    # Encode categorical features
    le_dict = {}
    for col in df.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        le_dict[col] = le

    # Scale numeric features
    scaler = MinMaxScaler()
    df_scaled = scaler.fit_transform(df)

    # Save scaler for later inference
    import joblib
    joblib.dump(scaler, 'scaler.save')

    return df_scaled, y, scaler


In [7]:
# (replace 'your_edge_iiot.csv' with your actual dataset path)
X_train, y_train, scaler = preprocess_edge_iiot(data_path)

# Select only benign samples for training
X_benign = X_train[y_train == 0]
print(f"Benign samples shape: {X_benign.shape}")


Benign samples shape: (1615643, 60)


In [13]:
# Hyperparameters
latent_dim = 100
input_dim = X_benign.shape[1]
batch_size = 128
epochs = 20

# Device (CPU/GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create Dataloader
from torch.utils.data import DataLoader, TensorDataset
dataset = TensorDataset(torch.tensor(X_benign, dtype=torch.float32))
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize Models
generator = Generator(latent_dim, input_dim).to(device)
discriminator = Discriminator(input_dim).to(device)

# Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002)
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002)

# Loss
adversarial_loss = nn.BCELoss()


In [14]:
for epoch in range(epochs):
    for real_samples, in dataloader:
        real_samples = real_samples.to(device)
        batch_size_curr = real_samples.size(0)

        # Labels
        valid = torch.ones(batch_size_curr, 1, device=device)
        fake = torch.zeros(batch_size_curr, 1, device=device)

        # Discriminator
        optimizer_D.zero_grad()
        real_pred = discriminator(real_samples)
        real_loss = adversarial_loss(real_pred, valid)

        z = torch.randn(batch_size_curr, latent_dim, device=device)
        fake_samples = generator(z)
        fake_pred = discriminator(fake_samples.detach())
        fake_loss = adversarial_loss(fake_pred, fake)

        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

        # Generator
        optimizer_G.zero_grad()
        z = torch.randn(batch_size_curr, latent_dim, device=device)
        gen_samples = generator(z)
        gen_pred = discriminator(gen_samples)
        g_loss = adversarial_loss(gen_pred, valid)
        g_loss.backward()
        optimizer_G.step()

    print(f"[Epoch {epoch+1}/{epochs}] D loss: {d_loss.item():.4f} | G loss: {g_loss.item():.4f}")


[Epoch 1/20] D loss: 0.2727 | G loss: 2.2476
[Epoch 2/20] D loss: 0.2255 | G loss: 2.1100
[Epoch 3/20] D loss: 0.3323 | G loss: 2.3597
[Epoch 4/20] D loss: 0.9703 | G loss: 1.5546
[Epoch 5/20] D loss: 0.3018 | G loss: 2.0592
[Epoch 6/20] D loss: 0.3876 | G loss: 2.1847
[Epoch 7/20] D loss: 0.4827 | G loss: 1.6665
[Epoch 8/20] D loss: 0.3438 | G loss: 1.5005
[Epoch 9/20] D loss: 0.4833 | G loss: 1.2605
[Epoch 10/20] D loss: 0.3724 | G loss: 1.3212
[Epoch 11/20] D loss: 0.3579 | G loss: 1.3788
[Epoch 12/20] D loss: 0.3026 | G loss: 1.3433
[Epoch 13/20] D loss: 0.3053 | G loss: 1.5657
[Epoch 14/20] D loss: 0.4404 | G loss: 1.4305
[Epoch 15/20] D loss: 0.4066 | G loss: 1.5029
[Epoch 16/20] D loss: 0.3196 | G loss: 1.4749
[Epoch 17/20] D loss: 0.3787 | G loss: 1.3809
[Epoch 18/20] D loss: 0.4819 | G loss: 1.8852
[Epoch 19/20] D loss: 0.3189 | G loss: 1.3946
[Epoch 20/20] D loss: 0.3467 | G loss: 1.3334


In [15]:
# Save the Generator
torch.save(generator.state_dict(), "generator.pth")
print("✅ Generator model saved successfully!")


✅ Generator model saved successfully!
